In [32]:
# Import some libraries we'll need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import scipy.sparse as sps
from sklearn.model_selection import train_test_split

In [33]:
business_data = pd.read_csv('../business.csv')
business_data.head(1)

,hours.Wednesday,attributes.RestaurantsDelivery,attributes.Open24Hours,attributes.DogsAllowed,attributes.CoatCheck,postal_code,attributes.Smoking,hours.Thursday,attributes.DietaryRestrictions,city,...,attributes.RestaurantsAttire,hours.Sunday,attributes.GoodForMeal,attributes.GoodForDancing,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,11:0-21:0,False,NaN,NaN,NaN,T2E 6L6,NaN,11:0-21:0,NaN,Calgary,...,casual,NaN,NaN,NaN,NaN,True,"{'BusinessParking': ""{'garage': False, 'street...",True,False,False


In [34]:
review_data = pd.read_csv('../review.csv')
review_data.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
0,0,msQe1u7Z_XuqjGoqhB0J5g,x7mDIiDB3jEiPGPHOmDzyw,The pizza was okay. Not the best I've had. I p...,iCQpiavjjPzJ5_3gPD5Ebg,2,2011-02-25,0,0
1,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0
2,1,msQe1u7Z_XuqjGoqhB0J5g,LZp4UX5zK3e-c5ZGSeo3kA,Terrible. Dry corn bread. Rib tips were all fa...,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,3,1
3,0,msQe1u7Z_XuqjGoqhB0J5g,Er4NBWCmCD4nM8_p1GRdow,Back in 2005-2007 this place was my FAVORITE t...,elqbBhBfElMNSrjFqW3now,2,2011-02-25,2,0
4,0,msQe1u7Z_XuqjGoqhB0J5g,jsDu6QEJHbwP2Blom1PLCA,Delicious healthy food. The steak is amazing. ...,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,0,0


In [35]:
restaurant_data = business_data[business_data['categories'].str.contains("Restaurants",na=False)]

In [48]:
restaurant_toronto = restaurant_data.loc[restaurant_data['city'] == 'Toronto']

In [49]:
review_rest_toronto = pd.merge(restaurant_toronto, review_data, on='business_id', how='left')

In [50]:
cols_required = ['attributes', 'categories', 'user_id', 'text', 'stars_y', 'business_id']
toronto_df = review_rest_toronto[cols_required]

In [51]:
toronto_df.head()

,attributes,categories,user_id,text,stars_y,business_id
0,"{'RestaurantsTableService': 'True', 'GoodForMe...","Cajun/Creole, Southern, Restaurants",ibftnXIQUhOvYhWCIyfRng,The food was fantastic. We tried the hush pupp...,4,tZnSodhPwNr4bzrwJ1CSbw
1,"{'RestaurantsTableService': 'True', 'GoodForMe...","Cajun/Creole, Southern, Restaurants",UkghXQuglpbzGpUhTKCBNg,Went here yesterday for a friend and my birthd...,4,tZnSodhPwNr4bzrwJ1CSbw
2,"{'RestaurantsTableService': 'True', 'GoodForMe...","Cajun/Creole, Southern, Restaurants",cu0ZyRh7GJyRqEeGKFDBkA,This restaurant continues to be one of my FAVO...,5,tZnSodhPwNr4bzrwJ1CSbw
3,"{'RestaurantsTableService': 'True', 'GoodForMe...","Cajun/Creole, Southern, Restaurants",uLQJAaULt91a263PsRyEqQ,We went to Southern Accent last night for a la...,4,tZnSodhPwNr4bzrwJ1CSbw
4,"{'RestaurantsTableService': 'True', 'GoodForMe...","Cajun/Creole, Southern, Restaurants",TfCpklkr3m5yobUwclxMEw,I'm from the deep south (about an hour from Ne...,5,tZnSodhPwNr4bzrwJ1CSbw


In [52]:
toronto_df.shape

(336912, 6)

In [59]:
user_ratings = toronto_df.groupby('user_id').agg(['count','mean']).reset_index()
user_ratings_400 = user_ratings[user_ratings['stars_y']['count'] > 400]
user_ratings_400.head()

user_id stars_y          
                                count      mean
17644  CxDOIDnH8gp9KXzpBHJYXw     934  3.246253
34270  Q9mA60HnY87C1TW5kjAZ6Q     470  3.961702
38637  TbhyP24zYZqZ2VJZgu1wrg     402  3.432836

In [62]:
highest_rated_user = toronto_df[toronto_df['user_id'] == 'CxDOIDnH8gp9KXzpBHJYXw']
highest_rated_user.head(5)

,attributes,categories,user_id,text,stars_y,business_id
355,"{'Alcohol': 'full_bar', 'HasTV': 'False', 'Noi...","Nightlife, Wine Bars, Indian, Restaurants, Bars",CxDOIDnH8gp9KXzpBHJYXw,I love the concept of Indian Street Food Compa...,3,cuXCQM-9VwpZlSneEY1b3w
1309,"{'BusinessParking': ""{'garage': False, 'street...","Irish, Specialty Food, Food, Restaurants",CxDOIDnH8gp9KXzpBHJYXw,Irish Potato Nachos is brand new take out food...,3,E9K1wSwr-3y7TIaVaoa-9g
1952,"{'RestaurantsTableService': 'True', 'GoodForMe...","Asian Fusion, Vietnamese, Restaurants",CxDOIDnH8gp9KXzpBHJYXw,I have been wanting to come and try Anh Dao Re...,3,oICXzFAaUMrYGzjRWmkw4Q
2210,"{'RestaurantsTableService': 'True', 'GoodForMe...","Restaurants, Seafood, Asian Fusion, Cajun/Creole",CxDOIDnH8gp9KXzpBHJYXw,2.5 Star for lunch Experience which offers Chi...,2,2VtTzskzFcwVnXgGYsY1lQ
2600,"{'RestaurantsTableService': 'False', 'GoodForM...","Bubble Tea, Food, Juice Bars & Smoothies, Food...",CxDOIDnH8gp9KXzpBHJYXw,Today Special's Honey Green Tea for $1.99 + $0...,3,kBYBXh7M9wKXvHgluJWiLQ


In [70]:
highest_rated_user['attributes'].value_counts()

{'RestaurantsTableService': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'breakfast': False, 'brunch': False}", 'Alcohol': 'beer_and_wine', 'Caters': 'False', 'HasTV': 'False', 'RestaurantsGoodForGroups': 'False', 'NoiseLevel': 'average', 'WiFi': 'no', 'RestaurantsAttire': 'casual', 'RestaurantsReservations': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'RestaurantsDelivery': 'False', 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'RestaurantsTakeOut': 'False', 'GoodForKids': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"}                                                                                                                                                                  

In [64]:
def clean_attributes(df):
    df_new = pd.DataFrame(df['attributes'].astype(str).str.split(',').tolist())
    df_new = df_new.apply(lambda x: x.str.lower()).apply(lambda x: x.str.replace('u\'','')).apply(lambda x: x.str.replace('u\"','')).apply(lambda x: x.str.replace(r'.*: {','')).apply(lambda x: x.str.replace('}\"','')).apply(lambda x: x.str.replace('\'','')).apply(lambda x: x.str.replace(':','')).apply(lambda x: x.str.replace('[','')).apply(lambda x: x.str.replace(']',''))
    return df_new.as_matrix()
    

In [65]:
def clean_categories(df):
    df_new = pd.DataFrame(df['categories'].astype(str).str.split(',').tolist())
    df_new = df_new.apply(lambda x: x.str.lower()).apply(lambda x: x.str.replace('u\'','')).apply(lambda x: x.str.replace('u\"','')).apply(lambda x: x.str.replace('\'','')).apply(lambda x: x.str.replace('[','')).apply(lambda x: x.str.replace(']',''))
    return df_new.as_matrix()

In [69]:
df1 = clean_categories(highest_rated_user)
df2 = clean_attributes(highest_rated_user)
print df2
features = np.concatenate((df1, df2),axis=1)
features = pd.DataFrame(features)
features.head()

/anaconda2/envs/yelplatest/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


[['{alcohol full_bar' ' hastv false' ' noiselevel average' ... None None
  None]
 ['{businessparking "{garage false' ' street false' ' validated false'
  ... None None None]
 ['{restaurantstableservice true' ' goodformeal "{dessert false'
  ' latenight false' ... None None None]
 ...
 ['{restaurantstableservice false' ' goodformeal "{dessert false'
  ' latenight false' ... None None None]
 ['{goodformeal "{dessert false' ' latenight false' ' lunch false' ...
  None None None]
 ['{restaurantstableservice true' ' goodformeal "{dessert false'
  ' latenight false' ... None None None]]


,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,nightlife,wine bars,indian,restaurants,bars,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,irish,specialty food,food,restaurants,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,asian fusion,vietnamese,restaurants,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,restaurants,seafood,asian fusion,cajun/creole,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,bubble tea,food,juice bars & smoothies,food stands,restaurants,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [18]:
restaurant_data['categories'].value_counts()

Restaurants, Pizza                                                                                                1092
Pizza, Restaurants                                                                                                1060
Restaurants, Mexican                                                                                               932
Mexican, Restaurants                                                                                               908
Restaurants, Chinese                                                                                               889
Chinese, Restaurants                                                                                               862
Restaurants, Italian                                                                                               523
Italian, Restaurants                                                                                               514
Restaurants, American (Traditional)             

In [19]:
restaurant_data.shape

(57173, 61)

In [20]:
cols = restaurant_data.columns
for col in cols:
    if "attribute" in col or "categories" in col:
        print restaurant_data[col].value_counts()
    else:
        restaurant_data.drop(col, axis = 1, inplace=True)

/anaconda2/envs/yelplatest/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


False    36341
True     12559
Name: attributes.RestaurantsDelivery, dtype: int64
False    251
True      26
Name: attributes.Open24Hours, dtype: int64
False    6304
True     1407
Name: attributes.DogsAllowed, dtype: int64
False    5010
True      557
Name: attributes.CoatCheck, dtype: int64
no         2594
outdoor    2207
yes         456
Name: attributes.Smoking, dtype: int64
{'dairy-free': False, 'gluten-free': False, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}     35
{'dairy-free': False, 'gluten-free': False, 'vegan': True, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': False}     19
{'dairy-free': False, 'gluten-free': True, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': False}     16
{'dairy-free': False, 'gluten-free': False, 'vegan': True, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}      12
{'dairy-free': False, 'gluten-free': False, 'vegan': False, 'kosher

False    3130
True       55
Name: attributes.BusinessAcceptsBitcoin, dtype: int64
True     5016
False    1112
Name: attributes.HappyHour, dtype: int64
True     20594
False     3135
Name: attributes.WheelchairAccessible, dtype: int64
True     18435
False    15745
Name: attributes.Caters, dtype: int64
no             756
yes_free       494
yes_corkage    159
Name: attributes.BYOBCorkage, dtype: int64
{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}     13487
{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}    10511
{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}                     9097
{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': Fal

False    464
True      57
Name: attributes.ByAppointmentOnly, dtype: int64
average      26919
quiet         9428
loud          3356
very_loud     1193
Name: attributes.NoiseLevel, dtype: int64
True     46439
False     2012
Name: attributes.BusinessAcceptsCreditCards, dtype: int64
False    4133
True     2621
Name: attributes.DriveThru, dtype: int64
True     22950
False    21393
Name: attributes.HasTV, dtype: int64
{'dj': False, 'background_music': False, 'no_music': False, 'karaoke': False, 'live': False, 'video': False, 'jukebox': False}    2755
{'dj': False, 'background_music': True, 'no_music': False, 'karaoke': False, 'live': False, 'video': False, 'jukebox': False}     1141
{'dj': False, 'background_music': False, 'no_music': False, 'karaoke': False, 'live': True, 'video': False, 'jukebox': False}      511
{'dj': False, 'background_music': False, 'no_music': False, 'karaoke': False, 'live': False, 'video': False, 'jukebox': True}      409
{'dj': True, 'background_music': False, 'no

{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfast': False, 'brunch': False}    16988
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'breakfast': False, 'brunch': False}       9859
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'breakfast': False, 'brunch': False}      5764
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'breakfast': False, 'brunch': False}      4485
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'breakfast': True, 'brunch': True}        1062
{'dessert': False, 'latenight': True, 'lunch': True, 'dinner': True, 'breakfast': False, 'brunch': False}         896
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfast': True, 'brunch': False}       895
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfast': False, 'brunch': True}       695
{'dessert': True, 'latenight': False, 'lunch': True, 'di

In [21]:
restaurant_data.head(5)

,attributes.RestaurantsDelivery,attributes.Open24Hours,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,attributes.DietaryRestrictions,attributes.GoodForKids,attributes.BYOB,attributes.AgesAllowed,attributes.Alcohol,...,attributes.WiFi,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,False,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,casual,NaN,NaN,NaN,True,"{'BusinessParking': ""{'garage': False, 'street...",True,False,False
1,False,NaN,True,NaN,NaN,NaN,True,NaN,NaN,none,...,no,casual,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,False,"{'RestaurantsTableService': 'False', 'GoodForM...",True,False,True
2,False,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,beer_and_wine,...,free,casual,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,True,"{'RestaurantsTableService': 'True', 'GoodForMe...",False,True,False
7,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,none,...,NaN,casual,NaN,NaN,NaN,NaN,"{'Alcohol': 'none', 'RestaurantsGoodForGroups'...",True,NaN,False
8,False,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,none,...,no,casual,"{'dessert': False, 'latenight': True, 'lunch':...",NaN,NaN,False,"{'RestaurantsTableService': 'False', 'GoodForM...",True,True,False


In [23]:
drop_cols = []
print 'PERCENTAGE OF MISSING VALUES'
for col in restaurant_data.columns:
    numofnull = restaurant_data[col].isnull().sum()
    missing = (float(numofnull)/float(restaurant_data.shape[0])) * 100
    if missing > 60:
        drop_cols.append(col)
    print col + ' = {} '.format(missing)

PERCENTAGE OF MISSING VALUES
attributes.RestaurantsDelivery = 14.4701170133 
attributes.Open24Hours = 99.5155055708 
attributes.DogsAllowed = 86.512864464 
attributes.CoatCheck = 90.2628863275 
attributes.Smoking = 90.8051003096 
attributes.DietaryRestrictions = 99.7603764014 
attributes.GoodForKids = 12.785755514 
attributes.BYOB = 98.4083396009 
attributes.AgesAllowed = 99.8775645847 
attributes.Alcohol = 23.414898641 
attributes.RestaurantsPriceRange2 = 10.6029069666 
attributes.RestaurantsTableService = 24.242212233 
attributes.HairSpecializesIn = 99.9877564585 
attributes.RestaurantsCounterService = 99.3056162874 
attributes.Corkage = 98.8508561734 
attributes.RestaurantsGoodForGroups = 12.1420950449 
categories = 0.0 
attributes.BusinessAcceptsBitcoin = 94.429188603 
attributes.HappyHour = 89.2816539276 
attributes.WheelchairAccessible = 58.4961432844 
attributes.Caters = 40.2165357774 
attributes.BYOBCorkage = 97.5355499974 
attributes.Ambience = 22.2710020464 
attributes.BestNi

In [24]:
print drop_cols

['attributes.Open24Hours', 'attributes.DogsAllowed', 'attributes.CoatCheck', 'attributes.Smoking', 'attributes.DietaryRestrictions', 'attributes.BYOB', 'attributes.AgesAllowed', 'attributes.HairSpecializesIn', 'attributes.RestaurantsCounterService', 'attributes.Corkage', 'attributes.BusinessAcceptsBitcoin', 'attributes.HappyHour', 'attributes.BYOBCorkage', 'attributes.BestNights', 'attributes.ByAppointmentOnly', 'attributes.DriveThru', 'attributes.Music', 'attributes.GoodForDancing', 'attributes.AcceptsInsurance']


In [28]:
x =  restaurant_data.loc[~restaurant_data['attributes.HairSpecializesIn'].isnull()]
restaurant_data.drop(drop_cols, axis = 1, inplace = True)
restaurant_data.drop(x.index, axis = 0, inplace = True)

In [29]:
restaurant_data.head()

,attributes.RestaurantsDelivery,attributes.GoodForKids,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.RestaurantsGoodForGroups,categories,attributes.WheelchairAccessible,attributes.Caters,attributes.Ambience,...,attributes.HasTV,attributes.BusinessParking,attributes.WiFi,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,False,True,NaN,2.0,NaN,True,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",NaN,NaN,NaN,...,True,"{'garage': False, 'street': True, 'validated':...",NaN,casual,NaN,True,"{'BusinessParking': ""{'garage': False, 'street...",True,False,False
1,False,True,none,2.0,False,True,"Chicken Wings, Burgers, Caterers, Street Vendo...",True,True,NaN,...,False,"{'garage': False, 'street': True, 'validated':...",no,casual,"{'dessert': False, 'latenight': False, 'lunch'...",False,"{'RestaurantsTableService': 'False', 'GoodForM...",True,False,True
2,False,True,beer_and_wine,2.0,True,True,"Breakfast & Brunch, Restaurants, French, Sandw...",NaN,False,"{'romantic': False, 'intimate': False, 'classy...",...,True,"{'garage': False, 'street': False, 'validated'...",free,casual,"{'dessert': False, 'latenight': False, 'lunch'...",True,"{'RestaurantsTableService': 'True', 'GoodForMe...",False,True,False
7,NaN,True,none,2.0,False,True,"Restaurants, Thai",NaN,NaN,NaN,...,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,casual,NaN,NaN,"{'Alcohol': 'none', 'RestaurantsGoodForGroups'...",True,NaN,False
8,False,True,none,1.0,False,True,"Mexican, Restaurants",True,False,"{'romantic': False, 'intimate': False, 'classy...",...,False,"{'garage': False, 'street': False, 'validated'...",no,casual,"{'dessert': False, 'latenight': True, 'lunch':...",False,"{'RestaurantsTableService': 'False', 'GoodForM...",True,True,False


In [31]:
restaurant_data['attributes.Ambience']

dtype('O')